# Дектораторы

В этом домашнем задании мы напишем собственные дектораторы, которые будут менять системные объекты. Но для начала мы с вами познакомимся с функцией `write`.

In [138]:
import sys

sys.stdout.write('Hello, my friend!')

Hello, my friend!

Это метод объектов file-like классов, то есть классов, которые реализуют семантику "Меня можно создать, из меня можно прочитать и в меня можно записать".

Самый главный пример такого объекта -- объект `file`, являющийся результатом вызова фукнции `open()`. Для простоты и универсальности взаимодействия, стандартный ввод и стандартный вывод тоже являются файлами, из которых можно читать и в которые можно писать. 

In [139]:
output = open("./some_test_data.txt", 'w')

In [140]:
output.write('123')

3

In [141]:
output.close()

Как вы могли заметить, функция возвращает число записанных байт. Это важная часть контракта, которую нужно поддержать, если вы хотите как-то подменять эту функцию.

## Задача 1

Для начала, давайте подменим метод `write` у объекта `sys.stdin` на такую функцию, которая перед каждым вызовом оригинальной функции записи данных в `stdout` допечатывает к тексту текущую метку времени.

In [142]:
original_write = sys.stdout.write

In [143]:
from datetime import datetime

def my_write(string_text: str) -> str:
    """Выводит текущее дату и время
    и заданный текст"""

    if string_text == '\n':
        original_write(string_text)
    else:
        original_write(f'[{datetime.strftime(datetime.now(), "%Y-%m-%d %H:%M:%S")}]: {string_text}')

sys.stdout.write = my_write

In [144]:
print('1, 2, 3')

[2022-12-06 00:30:06]: 1, 2, 3


In [145]:
sys.stdout.write = original_write

Вывод должен был бы быть примерно таким:

```
[2021-12-05 12:00:00]: 1, 2, 3
```

## Задача 2

Упакуйте только что написанный код в декторатор. Весь вывод фукнции должен быть помечен временными метками так, как видно выше.

In [146]:
from typing import Callable

In [147]:
def timed_output(function: Callable) -> Callable:
    def wrapper(*args, **kwargs) -> Callable: #произвольное количество аршументов
        sys.stdout.write = my_write
        function(*args, **kwargs)
        sys.stdout.write = original_write
    return wrapper

In [148]:
@timed_output
def print_greeting(name):
    print(f'Hello, {name}!')

In [149]:
print_greeting("Nikita")

[2022-12-06 00:30:10]: Hello, Nikita!


Вывод должен быть похож на следующий:

```
[2021-12-05 12:00:00]: Hello, Nikita!
```

## Задача 3

Напишите декторатор, который будет перенаправлять вывод фукнции в файл. 

Подсказка: вы можете заменить объект sys.stdout каким-нибудь другим объектом.

In [150]:
def redirect_output(filepath: str) -> Callable:
    def outer_wrapper(function: Callable) -> Callable:
        def inner_wrapper(*args,**kwargs):
            with open(filepath, 'w') as file:
                original_stdout=sys.stdout
                sys.stdout=file
                function(*args,**kwargs)
                sys.stdout = original_stdout
        return inner_wrapper
    return outer_wrapper

In [151]:
@redirect_output('./function_output.txt')
def calculate():
    for power in range(1, 5):
        for num in range(1, 20):
            print(num ** power, end=' ')
        print()

In [152]:
calculate()

In [153]:
%cat function_output.txt

UsageError: Line magic function `%cat` not found.
